#### Import packages

In [ ]:
import pandas as pd
import numpy as np
import pickle
import datetime as dt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_squared_error, r2_score, PredictionErrorDisplay
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import scipy.stats as stats
from sklearn.preprocessing import PolynomialFeatures  
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV, RepeatedKFold,cross_validate
import scipy as sp 
import statsmodels.api
from matplotlib.ticker import FormatStrFormatter

#### Pre-process demographic and social history

In [ ]:
demo = pd.read_csv(r'C:\Users\momenzadeha\Documents\hypogly_pred\EHR_extracts\meyer_2306_encounters_major_class_2023_05_30.csv')
SH = pd.read_excel(r'C:\Users\momenzadeha\Documents\hypogly_pred\EHR_extracts\Patient Info.xlsx')
with open(r'C:\Users\momenzadeha\Documents\mixed effects LR\LMM\MELRoutputCSNs.pkl', 'rb') as handle:
    CSN_list = pickle.load(handle)

In [ ]:
def prepare_static_inputs(demo, SH, CSN_list):
    demo_MRN = demo[['MRN', 'CSN', 'CONTACT_DATE', 'BMI', 'DPT_NAME']].drop_duplicates()
    SH_CSN = demo_MRN.merge(SH, on='MRN', how='inner')
    # Create a DataFrame from CSN_list and merge with SH_CSN on 'CSN'
    master_CSN = pd.DataFrame(CSN_list, columns=['CSN'])
    static_inputs = master_CSN.merge(SH_CSN, on='CSN', how='left')
    # Select and drop duplicate columns
    static_inputs = static_inputs[['CSN', 'MRN', 'CONTACT_DATE', 'BMI', 'BIRTH_DATE', 'ETHNIC_GROUP',
                                   'SEX', 'RACE_1', 'SMOKE_TOB_LAST_STATUS', 'ILL_DRUG_LAST_STATUS',
                                   'ALCOHOL_LAST_STATUS', 'DPT_NAME']].drop_duplicates()
    # Calculate age in years
    static_inputs['AGE'] = ((pd.to_datetime(static_inputs['CONTACT_DATE']) - pd.to_datetime(static_inputs['BIRTH_DATE'])) 
                            / np.timedelta64(1, 'D')) / 365
    static_inputs = static_inputs[['MRN', 'CSN', 'AGE', 'BMI', 'ETHNIC_GROUP', 'SEX', 'RACE_1',
                                   'SMOKE_TOB_LAST_STATUS', 'ILL_DRUG_LAST_STATUS', 'ALCOHOL_LAST_STATUS',
                                   'DPT_NAME']]
    static_inputs['MRN'] = static_inputs['MRN'].round().astype('Int64')
    return static_inputs


In [ ]:
static_inputs = prepare_static_inputs(demo, SH, CSN_list)

In [ ]:
def extract_admit_diag(demo):
    admit_diag = demo[['MRN', 'CSN', 'ADMIT_DIAG']]
    return admit_diag

In [ ]:
admit_diag = extract_admit_diag(demo)

In [ ]:
def classify_admission_diagnoses(admit_diag):
    # Define diagnosis patterns and corresponding column names
    diagnosis_dict = {
        'Admit_CHF': [
            'Acute on chronic congestive heart failure, unspecified heart failure type (HCC)',
            'Systolic CHF, acute on chronic (HCC)',
            'Congestive heart failure, unspecified HF chronicity, unspecified heart failure type (HCC)',
            'CHF',
            'Acute congestive heart failure, unspecified heart failure type (HCC)'
        ],
        'Admit_Sepsis': [
            'Sepsis, due to unspecified organism',
            'Sepsis, due to unspecified organism, unspecified whether acute organ dysfunction present (HCC)',
            'Sepsis',
            'Severe sepsis (HCC)',
            'Septic shock (HCC)'
        ],
        'Admit_GIB': [
            'gastrointestinal bleed',
            'GI bleed',
            'UGIB',
            'Gastrointestinal hemorrhage, unspecified gastrointestinal hemorrhage type'
        ],
        'Admit_NV': [
            'Nausea and vomiting, intractability of vomiting not specified, unspecified vomiting type',
            'Non-intractable vomiting with nausea, unspecified vomiting type',
            'Intractable vomiting with nausea, unspecified vomiting type'
        ],
        'Admit_AMS': [
            'Altered mental status, unspecified altered mental status type'
        ],
        'Admit_AKI': [
            'Acute kidney injury (HCC)',
            'Acute renal failure, unspecified acute renal failure type (HCC)',
            'Renal insufficiency',
            'Acute renal insufficiency'
        ],
        'Admit_ESRD': [
            'ESRD on dialysis (HCC)',
            'Chronic kidney disease, unspecified CKD stage',
            'ESRD',
            'ESRD on hemodialysis (HCC)'
        ]
    }
    
    # Iterate over diagnosis patterns and classify each diagnosis
    for column, patterns in diagnosis_dict.items():
        pattern = '|'.join(patterns)
        admit_diag[column] = np.where(admit_diag['ADMIT_DIAG'].str.contains(pattern, case=False, na=False), 1, 0)
    admit_diag['Admit_Pain'] = np.where(admit_diag['ADMIT_DIAG'].str.contains('pain', case=False, na=False), 1, 0)
    admit_diag.drop(['ADMIT_DIAG', 'MRN'], axis=1, inplace=True)
    admit_diag = admit_diag.groupby('CSN').sum()
    admit_diag[admit_diag > 1] = 1  
    admit_diag.reset_index(inplace=True)
    return admit_diag

In [ ]:
admit_diag=classify_admission_diagnoses(admit_diag)

#### Pre-process past medical history

In [ ]:
PMH = pd.read_csv(r'C:\Users\momenzadeha\Documents\hypogly_pred\EHR_extracts\Meyer2306 Diagnosis Hx.csv') 

In [ ]:
def update_pmh_with_icd10(pmh_df):
    # Define ICD-10 patterns and corresponding column names
    icd10_dict = {
        'PMH_Liver_failure': 'K72',
        'PMH_CKD': 'N18',
        'PMH_T1DM': 'E10',
        'PMH_T2DM': 'E11',
        'PMH_CHF': 'I50',
        'PMH_HypoThyroid': 'E03.0',
        'PMH_HyperThyroid': 'E05',
        'PMH_Pregnancy': 'Z33.1',
        'PMH_Anemia': 'D64.9',
        'PMH_Hypoglycemia': 'E16.2',
        'PMH_Malignancy': 'C80.1',
        'PMH_Pain': 'R52'
    }
    for column, pattern in icd10_dict.items():
        pmh_df[column] = pmh_df['CURRENT_ICD10_LIST'].str.contains(pattern, na=False).astype(int)
    pmh_df = (pmh_df[['MRN'] + list(icd10_dict.keys())]
              .groupby('MRN')
              .sum()
              .clip(upper=1))
    return pmh_df

In [ ]:
PMH = update_pmh_with_icd10(PMH)

#### Read in meds input previously made

In [ ]:
with open(r'/Users/momenzadeha/Documents/mixed effects LR/LMM/top500_DM_meds_24h.pkl', 'rb') as handle:
    top500_DM_meds = pickle.load(handle)

#### Read in labs input previously made

In [ ]:
with open(r'/Users/momenzadeha/Documents/mixed effects LR/LMM/sel_labs_24h.pkl', 'rb') as handle:
    sel_labs = pickle.load(handle)

In [ ]:
# Clean and rename columns in the DataFrame `sel_labs`
def clean_and_rename_columns(df):
    new_columns = []
    for col in df.columns:
        parts = col.split(',')
        if len(parts) >= 2:
            # Extract and clean the second part of the split column name
            second_part = parts[1].strip().replace("(", "").replace(")", "").replace(" ", "").replace("'", "")
            new_columns.append(second_part)
        else:
            # Keep the original column name if it doesn't split into at least two parts
            new_columns.append(col.replace("'", ""))
    df.columns = new_columns
    df = df[['CSN', 'CREATININE_mean', 'GLUCOSE-POC_mean', 'GLUCOSE-POC_min', 'GLUCOSE-POC_last', 'GLUCOSE-POC_max']]
    df = df.set_index('CSN')
    # Remove rows with NaN values
    df = df.dropna()
    return df

In [ ]:
sel_labs = clean_and_rename_columns(sel_labs)

#### read in output BG previously made

In [ ]:
with open(r'/Users/momenzadeha/Documents/mixed effects LR/LMM/outputBG_df.pkl', 'rb') as handle:
    outputBG = pickle.load(handle)

In [ ]:
# Rename and select relevant columns in `outputBG`
outputBG = outputBG.rename(columns={'PAT_ENC_CSN_ID': 'CSN'})[['CSN', 'ORD_VALUE']]
# Merge with `static_inputs` and filter out rows with missing MRN
static_output_df = outputBG.merge(static_inputs, on='CSN', how='inner').dropna(subset=['MRN']).reset_index(drop=True)
# Perform successive merges to combine data
med_lab_static = top500_DM_meds.merge(static_output_df, on='CSN', how='inner')
med_lab_static_PMH = med_lab_static.merge(sel_labs, on='CSN', how='inner')
med_lab_static_PMH_admdx = med_lab_static_PMH.merge(PMH, on='MRN', how='inner').merge(admit_diag, on='CSN', how='inner')

### one hot encode

In [ ]:
categorical_columns = ['ETHNIC_GROUP', 'SEX', 'RACE_1', 'SMOKE_TOB_LAST_STATUS', 
                        'ILL_DRUG_LAST_STATUS', 'ALCOHOL_LAST_STATUS']
# Replace NaN values with the string 'Missing' for the selected columns
med_lab_static_PMH_admdx[categorical_columns] = med_lab_static_PMH_admdx[categorical_columns].fillna('Missing')
# One-hot encode the categorical columns
med_lab_static_PMH_admdx = pd.get_dummies(med_lab_static_PMH_admdx, columns=categorical_columns)

In [ ]:
#Remove missing BMIs (0.7% of encounters)
med_lab_static_PMH_admdx=med_lab_static_PMH_admdx.fillna(0)

In [ ]:
# Define OR departments 
OR_depts = [
    '6-ACU (6-PACU)', '8-ACU (8-PACU)', '3-ACU (3-PACU)', '7GI-ACU (GI LAB PACU)', '6-PACU', '7-PACU', '5-PACU', '7-ACU (7-PACU)',
    'GI LAB MAIN', '8-OR', '6-OR', '5-OR', 'CARDIAC CATH LAB MAIN', 'AHSP 5-PACU', '3-PACU', 'INTERVENTION RAD MAIN', '8-PRE-OP', 
    '8-PACU', '7-OR', '3-PREOP', 'RT BRONCHOSCOPY MAIN', '5-ACU (5-PACU)'
]
# Map for ICU and non-ICU replacement
dept_replacement_map = {
    'non_ICU': [
        '6-NE', '6-NW', '4-NW', '5-NW', '5-NE', '5-SE', '7-SE', '5-SW', '6-SE', '8-NE', '6-SW', '3-N', '7-NE', '7-NW', '4-SE',
        '7-SW', '4-SW', '8-NW', '3N-UNIV', '8-SE', '8-SW', '3S-UNIV', '3SPT', 'MDRH 1 SOUTH', 'MDRH 1 NORTH', '3-SW', '3-SE',
        '3-SW OB', '3-NW', '7 REHAB', '6-ACU (6-PACU)', 'MDRH MED SSU', '8-ACU (8-PACU)', '4S-MON', 'MDRH 1 EAST', '3-ACU (3-PACU)',
        '7GI-ACU (GI LAB PACU)', '3-N MFCU', '3-NE', '6-PACU', '4-NE', 'OLD PEDS 4-NE', 'AHSP 5-ACU', '7-PACU', '5-PACU', '7-ACU (7-PACU)',
        'GI LAB MAIN', '8-OR', '3-SE OB', '6-OR', '5-OR', 'AHSP 5-OSU', 'CARDIAC CATH LAB MAIN', 'AHSP 5-PACU', '3-PACU', 
        'MDRH EMERGENCY DEPT', '2-SCCT PEDS', 'INTERVENTION RAD MAIN', '8-PRE-OP', '8-PACU', '7-OR', '3-PREOP', '3-LDR',
        'RT BRONCHOSCOPY MAIN', 'DIALYSIS ACUTE MAIN', '5-ACU (5-PACU)'
    ],
    'ICU': [
        '6-ICU', '8S-NSICU', '8N-ICU', '7S-RICU', '7N-MICU', '4N-CICU', '5S-SICU', '5N-SICU', '5N-SICU', '6S-CSICU', '6N-CSICU',
        '4S-ICU', 'MDRH INTENSIVE CARE', '4S-PICU'
    ]
}
# Function to categorize departments
def categorize_dept(dept_name):
    if dept_name in OR_depts:
        return 'OR'
    elif dept_name in dept_replacement_map['non_ICU']:
        return 'non_ICU'
    elif dept_name in dept_replacement_map['ICU']:
        return 'ICU'
    else:
        return 'Other'
# Apply the function to create admit_dept column
med_lab_static_PMH_admdx['admit_dept'] = med_lab_static_PMH_admdx['DPT_NAME'].apply(categorize_dept)
# One-hot encode the admit_dept column
one_hot = pd.get_dummies(med_lab_static_PMH_admdx['admit_dept'], prefix='admit_dept')
# Drop the original admit_dept column and join the one-hot encoded columns
med_lab_static_PMH_admdx = med_lab_static_PMH_admdx.drop('admit_dept', axis=1).join(one_hot)
# Replace DPT_NAME values
med_lab_static_PMH_admdx['DPT_NAME'] = med_lab_static_PMH_admdx['DPT_NAME'].replace({
    '6-ICU': 'ICU', '8S-NSICU': 'ICU', '8N-ICU': 'ICU', '7S-RICU': 'ICU', '7N-MICU': 'ICU', '4N-CICU': 'ICU',
    '5S-SICU': 'ICU', '5N-SICU': 'ICU', '6S-CSICU': 'ICU', '6N-CSICU': 'ICU', '4S-ICU': 'ICU', 'MDRH INTENSIVE CARE': 'ICU',
    '4S-PICU': 'ICU'
})
# One-hot encode the updated DPT_NAME column
one_hot = pd.get_dummies(med_lab_static_PMH_admdx['DPT_NAME'], prefix='DPT_NAME')
med_lab_static_PMH_admdx = med_lab_static_PMH_admdx.drop('DPT_NAME', axis=1).join(one_hot)
med_lab_static_PMH_admdx = med_lab_static_PMH_admdx.replace({False: 0, True: 1})

In [ ]:
# Replace special characters in column names with underscores
special_chars = [' ', '.', '-', '/', '(', ')', '%', ',', '+', '&', '=']
for char in special_chars:
    med_lab_static_PMH_admdx.columns = med_lab_static_PMH_admdx.columns.str.replace(char, '_')

In [ ]:
med_lab_static_PMH_admdx_cleaned=med_lab_static_PMH_admdx.copy()

#### Make descriptive statistics tables

In [ ]:
for x in med_lab_static_PMH_admdx_cleaned[['AGE','BMI','CREATININE_mean','GLUCOSE_POC_mean','GLUCOSE_POC_min','GLUCOSE_POC_last','GLUCOSE_POC_max']].columns:
    print('**',x)
    median=med_lab_static_PMH_admdx_cleaned[x].median()
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)  
    Q1 = med_lab_static_PMH_admdx_cleaned[x].quantile(0.25)  
    Q3 = med_lab_static_PMH_admdx_cleaned[x].quantile(0.75)  
  
    # Calculate the IQR  
    IQR = Q3 - Q1  
    print(f"median: {median}")   
    print(f"Q1: {Q1}")  
    print(f"Q3: {Q3}")    
# Continuous variables to analyze
continuous_vars = ['AGE', 'BMI', 'CREATININE_mean', 'GLUCOSE_POC_mean', 
                   'GLUCOSE_POC_min', 'GLUCOSE_POC_last', 'GLUCOSE_POC_max']
# Loop through each continuous variable 
for var in med_lab_static_PMH_admdx_cleaned[continuous_vars].columns:
    print(f"** {var} **")  
    # Calculate the median value of the variable
    median = med_lab_static_PMH_admdx_cleaned[var].median()
    # Calculate the 25th percentile (Q1) and 75th percentile (Q3)
    Q1 = med_lab_static_PMH_admdx_cleaned[var].quantile(0.25)
    Q3 = med_lab_static_PMH_admdx_cleaned[var].quantile(0.75)
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1
    # Output the statistical values
    print(f"Median: {median}")
    print(f"Q1 (25th percentile): {Q1}")
    print(f"Q3 (75th percentile): {Q3}")
    print(f"IQR (Interquartile Range): {IQR}")
    print('-' * 40) 

In [ ]:
# Categorical variables: Loop through each column from the specified range
categorical_vars = med_lab_static_PMH_admdx_cleaned.iloc[:, 525:].columns
# Loop through each categorical variable
for var in categorical_vars:
    print(f"** {var} **")  # Variable name header
    count = med_lab_static_PMH_admdx_cleaned[var].value_counts().get(1, 0)
    percent = (count / med_lab_static_PMH_admdx_cleaned.shape[0]) * 100
    # Output the count and percentage
    print(f"Count of '1': {count}")
    print(f"Percentage: {percent:.2f}%")
    print('-' * 40)  # Divider for readability

#### Feature extract with Lasso 

In [ ]:
# Define the target variable (y) and feature set (X)
y = med_lab_static_PMH_admdx_cleaned['ORD_VALUE']  # Target variable
X = med_lab_static_PMH_admdx_cleaned.drop(['ORD_VALUE', 'CSN', 'MRN'], axis=1)  # Feature set, excluding non-relevant columns
# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Further split the training data into training and validation sets (80% of training set for training, 20% for validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Define a list of alpha values to test, ranging from 10^-4 to 10^4
alphas = np.logspace(-4, 4, 100)  # 100 values on a log scale between 10^-4 and 10^4
# Initialize and fit the LassoCV model with 5-fold cross-validation
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42)  # 5-fold CV ensures a robust model
lasso_cv.fit(X_train, y_train)
# Retrieve the best alpha value found during cross-validation
best_alpha = lasso_cv.alpha_
print(f'The optimal alpha value found: {best_alpha}')
# Extract the list of alpha values and the corresponding MSE path for each fold
alphas = lasso_cv.alphas_  # Alpha values explored by LassoCV
mse_path = lasso_cv.mse_path_  # Mean squared error for each fold and alpha
# Plot MSE vs. log(alpha) for each fold
plt.figure(figsize=(7, 5))
for i in range(mse_path.shape[1]):
    plt.plot(np.log10(alphas), mse_path[:, i], label=f'Fold {i+1}')  # Log scale of alpha on x-axis
# Customize the plot 
plt.xlabel('Log10(alpha)', fontsize=30) 
plt.ylabel('MSE', fontsize=30)          
plt.tick_params(axis='x', labelsize=25)  
plt.tick_params(axis='y', labelsize=25)  
plt.legend(fontsize=20)                  
plt.tight_layout()                       
# Save the figure
# plt.savefig('figures/MSEvalpha.svg', dpi=300)
plt.show()

In [ ]:
# Use the fitted lasso_cv model to predict the target variable on the validation data
y_pred = lasso_cv.predict(X_val)
# Evaluate the model performance using the R^2 score
r2_lasso = lasso_cv.score(X_val, y_val)  
print(f'R^2 score on validation data: {r2_lasso}')  
# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_val, y_pred)
print(f'MSE on validation data: {mse}') 
# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse) 
print(f'RMSE on validation data: {rmse}') 

In [ ]:
_, ax = plt.subplots(figsize=(7,5.5))
display = PredictionErrorDisplay.from_predictions(
    y_val, y_pred, kind="actual_vs_predicted", ax=ax, scatter_kwargs={"color": "skyblue", "edgecolor": "black", "alpha": 0.5}
)
plt.xlabel('Predicted', fontsize=35)
plt.ylabel('Actual', fontsize=35)
plt.tick_params(axis='y', labelsize=30)  
plt.tick_params(axis='x', labelsize=30)  
plt.tight_layout()
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
#plt.savefig('figures/lasso_actual_vs_predicted.svg', dpi=300,bbox_inches='tight')

In [ ]:
# Get the coefficients from the model  
coefficients = lasso_cv.coef_  
# Pair feature names with their corresponding coefficients  
feature_coefficients = list(zip(X_train.columns, coefficients))  
# Sort features by the absolute values of their coefficients in descending order  
feature_coefficients = sorted(feature_coefficients, key=lambda x: abs(x[1]), reverse=True)  
# Separate the feature names and their corresponding coefficients for plotting  
sorted_features, sorted_coefficients = zip(*feature_coefficients)  
plt.figure(figsize=(4,3))
plt.hist(coefficients,bins=50, log=True,color='skyblue', edgecolor='black', linewidth=0.5)
plt.xlabel('Lasso Coefficients', fontsize=18)
plt.ylabel('Logged Counts', fontsize=18)
plt.tick_params(axis='both', labelsize=15)  
plt.tight_layout()
#plt.savefig('figures/lasso_coeff_hist.svg', dpi=300)
plt.show()  

In [ ]:
# Perform cross-validation 
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=0)   
cv_model = cross_validate(  
    Lasso(alpha=lasso_cv.alpha_, max_iter=10000, random_state=42),  
    X_train, y_train,
    cv=cv,  
    return_estimator=True,  
    n_jobs=-1,  
)  

In [ ]:
# Extract the coefficients from each fold and repeat  
coefs = [est.coef_ for est in cv_model['estimator']]  
coefs_df = pd.DataFrame(coefs, columns=X_train.columns) 
# Calculate the mean absolute coefficient value for each feature  
mean_coefs = coefs_df.abs().mean().sort_values(ascending=False)  
# Select the names of the top 25 features with the highest mean coefficient value  
top_25_feature_names = mean_coefs.head(25).index  
# Subset the original DataFrame to include only the top 25 features  
top_25_coefs_df = coefs_df[top_25_feature_names]  
mean_coefs_df=pd.DataFrame(mean_coefs)
mean_coefs_df.columns = ['mean absolute value coefficient']
#select top 50 features 
top_50_feature_names = mean_coefs.head(50).index  
# Plot the coefficient variability for the top 25 features using a boxplot  
plt.figure(figsize=(9, 6))  
sns.boxplot(data=top_25_coefs_df, orient="h", color="skyblue")  
plt.axvline(x=0, color="grey", linestyle="--")  
plt.xlabel("Coefficient value", fontsize=22)  
plt.ylabel("Features",fontsize=22)  
plt.tick_params(axis='both', labelsize=13) 
plt.tight_layout()
#plt.savefig('figures/coeff_var_lasso.svg', dpi=300)
plt.show()  

#### Linear mixed effects model

In [ ]:
top_50_feature_names = top_50_feature_names.to_list()
top_50_feature_names.append('MRN')  # 'MRN' (Medical Record Number) is often used as an identifier for patients in the dataset
top_50_feature_names.append('ORD_VALUE')  # 'ORD_VALUE' is the target variable, representing the outcome to be predicted
# Select the top 50 features along with 'MRN' and 'ORD_VALUE' from the cleaned DataFrame
inputs_lasso = med_lab_static_PMH_admdx_cleaned[top_50_feature_names]  # Subset the DataFrame using the list of selected features
# Define the feature matrix 'X' and the target variable 'y' for model training
X_train, X_test, y_train, y_test = train_test_split(
    inputs_lasso, y,  
    test_size=0.2,  
    random_state=42
)

In [ ]:
# Fit LMM Using training set
column_list = X_train.columns.to_list()
column_list = [col for col in column_list if col not in ['MRN', 'ORD_VALUE']]
formula = 'ORD_VALUE ~ ' + ' + '.join(column_list)
md = smf.mixedlm(formula, X_train, groups=X_train["MRN"])
mdf = md.fit()
print(mdf.summary())

In [ ]:
# Evaluate on test set and report metrics
y_pred = mdf.predict(X_test)
r_squared = r2_score(y_test, y_pred)
print("R-squared:", r_squared)
mean_squared_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print("RMSE:", np.sqrt(mse))

In [ ]:
# Perform multiple testing correction using Benjamini-Hochberg 
adj_p = statsmodels.stats.multitest.multipletests(
    pvals=mdf.pvalues.to_list(), 
    alpha=0.01,                  
    method="fdr_bh"               
)
summary_frame = mdf.summary().tables[1]
dict = {
    'variable': mdf.pvalues.index,  
    'unadjusted_p_value': mdf.pvalues.to_list(),  
    'adjusted_p_value': adj_p[1],  # Adjusted p-values after correction
    'coefficient': summary_frame['Coef.'], 
    'standard error': summary_frame['Std.Err.'],  
    '95CI_lower': summary_frame['[0.025'],  
    '95CI_upper': summary_frame['0.975]']
}
df = pd.DataFrame(dict).set_index('variable')

In [ ]:
sig_24h=df[df['adjusted_p_value']<0.05].sort_values(by='adjusted_p_value').reset_index().variable.to_list()
items_to_remove = {'Group Var','Intercept'}
sig_24h = [item for item in sig_24h if item not in items_to_remove]